# 🚀 Focoos HUB Integration

🐍 Setup Focoos

In [ ]:
%pip install 'focoos @ git+https://github.com/FocoosAI/focoos.git'

## Get User Info

In [ ]:
from focoos.hub import FocoosHUB

hub = FocoosHUB()
user_info = hub.get_user_info()
user_info

## List Remote Models

In [ ]:
from focoos.hub import FocoosHUB

hub = FocoosHUB()
models = hub.list_remote_models()
models

## List Remote Datasets

In [ ]:
from focoos.hub import FocoosHUB

hub = FocoosHUB()
datasets = hub.list_remote_datasets(include_shared=True)
datasets

## Get Model Info

In [ ]:
from focoos.hub import FocoosHUB

hub = FocoosHUB()
model_info = hub.get_model_info("8e1f8b58d85b4771")
model_info

## Remote Inference

In [ ]:
from io import BytesIO

import requests
from PIL import Image

from focoos.hub import FocoosHUB
from focoos.utils.vision import annotate_image

response = requests.get("https://public.focoos.ai/samples/pexels-abby-chung.jpg")
image = Image.open(BytesIO(response.content))

hub = FocoosHUB()
model = hub.get_remote_model("8e1f8b58d85b4771")
results = model.infer(image=image, threshold=0.5)

annotated_image = annotate_image(
    im=image, detections=results, task=model.model_info.task, classes=model.model_info.classes
)

display(image)
display(annotated_image)

In [ ]:
from focoos.model_manager import ModelManager

model = ModelManager.get("hub://8e1f8b58d85b4771")

model